In [1]:
import pandas as pd
import cohere
import time
from annoy import AnnoyIndex
co = cohere.Client('bE6Is3wvtmXyHtgnCQocDIgdH7PcYwdR21ZhnXgN') 

def embeddings(texts,sleep_time=5):
    # add a wait time to simulate a long running process
    time.sleep(sleep_time)
    response = co.embed(
        model='large',
        texts=list(texts), 
        truncate='LEFT').embeddings
    return response

df = pd.read_excel('cohere_docs_embeddings.xlsx')

def load_data(df,):
    df['embeddings'] = embeddings(df['text'])
    # drop rows frm text_df that havve less than 8 words
    df = df[df['text'].str.split().str.len() > 10]
    # Create the search index, pass the size of embedding
    search_index = AnnoyIndex(4096, 'angular')
    # Add all the vectors to the search index, these are stored in the dataframe 'post_members['embeddings']'
    for i, vector in enumerate(df['embeddings']):
        search_index.add_item(i, vector)
    # Build the search index
    search_index.build(10)
    #save the search index
    search_index.save('search_index.ann')
    return df, search_index

df, search_index = load_data(df)

ValueError: Length of values (10) does not match length of index (69)

In [14]:
def search(query, n_results, df, search_index, co):
    # Get the query's embedding
    query_embed = co.embed(texts=[query],
                    model="large",
                    truncate="LEFT").embeddings

    # Get the nearest neighbors and similarity score for the query and the embeddings, append it to the dataframe
    nearest_neighbors = search_index.get_nns_by_vector(query_embed[0], n_results, include_distances=True)
    # filter the dataframe to only include the nearest neighbors using the index
    df = df[df.index.isin(nearest_neighbors[0])]
    df['similarity'] = nearest_neighbors[1]
    df['nearest_neighbors'] = nearest_neighbors[0]
    df = df.sort_values(by='similarity', ascending=False)
    return df


# run function to search for the query
test = search('What are usage examples for the generate endpoint?', 5, df, search_index, co)
test

C:\Users\Elle\AppData\Local\Temp\ipykernel_14488\1859625581.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['similarity'] = nearest_neighbors[1]
C:\Users\Elle\AppData\Local\Temp\ipykernel_14488\1859625581.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nearest_neighbors'] = nearest_neighbors[0]


,text,title,link,embeddings,similarity,nearest_neighbors
68,['This endpoint classifies text into one of se...,Toxicity Detectionpost,/reference/toxicity-detection,"[0.0592041, -0.49951172, -0.72558594, 0.599121...",0.985209,12
43,['This endpoint generates realistic text condi...,Co.Generatepost,/reference/generate,"[2.8222656, -0.94677734, 1.5234375, -0.8032226...",0.959930,67
67,['This endpoint classifies text into one of se...,/classify,/reference/toxicity-detection,"[0.0592041, -0.49951172, -0.72558594, 0.599121...",0.959930,68
12,"['When you call the Generate endpoint, you hav...",Number of Generations,/docs/number-of-generations,"[1.7412109, -0.23046875, 0.6401367, -0.7202148...",0.880471,42
42,['This endpoint generates realistic text condi...,/generate,/reference/generate,"[2.8222656, -0.94677734, 1.5234375, -0.8032226...",0.880471,43
